# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [15]:
# Dependencies and Setup
import hvplot
import hvplot.pandas
import pandas as pd
import requests
import geopandas as gpd
import holoviews as hv
from pprint import pprint

# Import API key
from api_keys import geoapify_key

In [16]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,dongyang,29.2678,120.2253,42.93,53,3,5.79,CN,1699831828
1,1,blackmans bay,-43.0167,147.3167,61.52,49,7,6.22,AU,1699831252
2,2,vila velha,-20.3297,-40.2925,80.94,84,0,9.22,BR,1699831256
3,3,waitangi,-43.9535,-176.5597,54.57,86,56,19.37,NZ,1699831242
4,4,amal,59.0510,12.7049,31.39,84,25,8.39,SE,1699831829


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [29]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    frame_width = 800,
    frame_height = 600
)

map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [18]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df.loc[(city_data_df['Max Temp'] <= 90) & (city_data_df['Max Temp'] >= 50) & (city_data_df['Humidity'] <= 50) & (city_data_df['Wind Speed'] <= 10)]


# Drop any rows with null values
ideal_weather = ideal_weather.dropna()

# Display sample data
ideal_weather.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,blackmans bay,-43.0167,147.3167,61.52,49,7,6.22,AU,1699831252
16,16,gombi,10.1614,12.7399,75.43,33,78,3.38,NG,1699831833
45,45,naracoorte,-36.9579,140.7383,63.75,43,31,8.21,AU,1699831840
46,46,luziania,-16.2525,-47.9503,87.49,28,55,4.88,BR,1699831840
58,58,as sulayyil,20.4607,45.5779,66.45,45,24,8.19,SA,1699831844


In [19]:
### Step 3: Create a new DataFrame called `hotel_df`.

In [20]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
1,blackmans bay,AU,-43.0167,147.3167,49,
16,gombi,NG,10.1614,12.7399,33,
45,naracoorte,AU,-36.9579,140.7383,43,
46,luziania,BR,-16.2525,-47.9503,28,
58,as sulayyil,SA,20.4607,45.5779,45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [22]:
params = {
    "filter": "hotel",
    "apiKey": geoapify_key
}
base_url = "https://api.geoapify.com/v2/places"
name_address = requests.get(base_url, params=params)
name_address = name_address.json()

pprint(name_address)

{'error': 'Bad Request',
 'message': '"filter[0]" does not match any of the allowed types',
 'statusCode': 400}


In [28]:
# Set parameters to search for a hotel
radius = 10000
categories = f"accommodation.hotel"

params = {
    "categories":categories,
    "apiKey":geoapify_key
}
 
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
blackmans bay - nearest hotel: Villa Howden
gombi - nearest hotel: No hotel found
naracoorte - nearest hotel: Kincraig Hotel /Smith & Jones Cafe
luziania - nearest hotel: No hotel found
as sulayyil - nearest hotel: No hotel found
itapolis - nearest hotel: No hotel found
mailsi - nearest hotel: No hotel found
alice springs - nearest hotel: Aurora Alice Springs
bechar - nearest hotel: Hôtel Madania
castelo do piaui - nearest hotel: No hotel found
fallon - nearest hotel: Value Inn
port hueneme - nearest hotel: No hotel found
jisr ash shughur - nearest hotel: No hotel found
victorica - nearest hotel: Hotel Mazamet
passos - nearest hotel: Hotel Maldi
ar raqqah - nearest hotel: No hotel found
tari - nearest hotel: No hotel found
kankan - nearest hotel: Buffet de la gare
pahrump - nearest hotel: Holiday Inn Express & Suites
filingue - nearest hotel: No hotel found
tazacorte - nearest hotel: App Leyma
firminopolis - nearest hotel: Pousada Farol
naliya - nearest hotel: No 

,City,Country,Lat,Lng,Humidity,Hotel Name
1,blackmans bay,AU,-43.0167,147.3167,49,Villa Howden
16,gombi,NG,10.1614,12.7399,33,No hotel found
45,naracoorte,AU,-36.9579,140.7383,43,Kincraig Hotel /Smith & Jones Cafe
46,luziania,BR,-16.2525,-47.9503,28,No hotel found
58,as sulayyil,SA,20.4607,45.5779,45,No hotel found
78,itapolis,BR,-21.5956,-48.8128,25,No hotel found
84,mailsi,PK,29.8003,72.1758,42,No hotel found
110,alice springs,AU,-23.7000,133.8833,47,Aurora Alice Springs
131,bechar,DZ,31.6167,-2.2167,34,Hôtel Madania
134,castelo do piaui,BR,-5.3222,-41.5525,28,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [34]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    frame_width = 800,
    frame_height = 600,
    hover_cols = ["City", "Country", "Humidity", "Hotel Name"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)